<a href="https://colab.research.google.com/github/audi0417/LineBot-Medical-Assistant-A-LIANG/blob/main/KSPM25_CNN-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt #圖表繪製套件
import datetime #時間處理套件
from sklearn.preprocessing import MinMaxScaler  #最大值及最小值標準化
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM,BatchNormalization
from keras.callbacks import EarlyStopping
from keras import callbacks
from  sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from math import sqrt
from keras.layers import Conv1D,  Bidirectional, TimeDistributed, MaxPooling1D, Flatten,Conv2D,MaxPool2D,Input
from sklearn.model_selection import train_test_split

In [ ]:
#df= pd.read_csv('/content/KSPM25.csv')
df= pd.read_excel('/content/KSPM25.xlsx')
df

,NO,Day,Hour,Func_Date,PM25,PM10,so2,no2,WIND_SPEED,CO,O3
0,1,2016-01-01,0,2016/1/1_0,84.750,140.750,6.3000,31.6250,1.4875,0.80750,7.9625
1,2,2016-01-01,1,2016/1/1_1,89.375,139.875,4.7125,31.3750,1.2750,0.78750,5.5750
2,3,2016-01-01,2,2016/1/1_2,81.375,130.750,4.3125,29.7500,1.3375,0.75250,5.2000
3,4,2016-01-01,3,2016/1/1_3,74.750,124.125,4.8125,29.2500,1.5875,0.74125,4.4375
4,5,2016-01-01,4,2016/1/1_4,74.750,117.000,4.7750,29.5000,1.3375,0.72250,3.3500
...,...,...,...,...,...,...,...,...,...,...,...
26299,26300,2018-12-31,19,2018/12/31_19,28.750,71.125,3.2750,22.2500,2.0750,0.50500,19.0000
26300,26301,2018-12-31,20,2018/12/31_20,24.250,63.750,2.9500,21.2500,1.7250,0.48375,15.8250
26301,26302,2018-12-31,21,2018/12/31_21,20.375,56.375,2.9000,21.7500,1.5250,0.49250,12.7625
26302,26303,2018-12-31,22,2018/12/31_22,18.875,50.875,2.7625,20.9250,1.6875,0.49250,10.9875


In [ ]:
#df.drop(['Func_Date','PM10', 'so2', 'no2', 'WIND_SPEED','CO','O3'], axis=1, inplace=True) 
df.drop(['Func_Date','NO','Hour','Day'], axis=1, inplace=True) 
df

,PM25,PM10,so2,no2,WIND_SPEED,CO,O3
0,84.750,140.750,6.3000,31.6250,1.4875,0.80750,7.9625
1,89.375,139.875,4.7125,31.3750,1.2750,0.78750,5.5750
2,81.375,130.750,4.3125,29.7500,1.3375,0.75250,5.2000
3,74.750,124.125,4.8125,29.2500,1.5875,0.74125,4.4375
4,74.750,117.000,4.7750,29.5000,1.3375,0.72250,3.3500
...,...,...,...,...,...,...,...
26299,28.750,71.125,3.2750,22.2500,2.0750,0.50500,19.0000
26300,24.250,63.750,2.9500,21.2500,1.7250,0.48375,15.8250
26301,20.375,56.375,2.9000,21.7500,1.5250,0.49250,12.7625
26302,18.875,50.875,2.7625,20.9250,1.6875,0.49250,10.9875


In [ ]:
split_point = int(len(df)*0.9) # 訓練集+驗證集 : 測試集
data1 = df.iloc[:split_point].copy()
test = df.iloc[split_point:].copy()

split_point = int(len(data1)*0.8) # 訓練集+驗證集 : 測試集
train = data1.iloc[:split_point].copy()
val = data1.iloc[split_point:].copy()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
train = scaler.fit_transform(train)
val = scaler.fit_transform(val) 
test = scaler.fit_transform(test)

In [ ]:
def dataset(timesteps,features):
  x_train = []
  y_train = []

  for i in np.arange(timesteps, len(train)):
      x_train.append(train[i - timesteps:i, :])
      y_train.append(train[i, features-1])

  # (樣本數，Timesteps,特徵數量)
  x_train, y_train = np.array(x_train), np.array(y_train)

  x_val = []
  y_val = []


  for i in np.arange(timesteps, len(val)):
      x_val.append(val[i - timesteps:i, :])
      y_val.append(val[i, features-1])


  x_val, y_val = np.array(x_val), np.array(y_val)


  x_test = []
  y_test = []


  for i in np.arange(timesteps, len(test)):
      x_test.append(test[i - timesteps:i, :])
      y_test.append(test[i, features-1])


  x_test, y_test = np.array(x_test), np.array(y_test)

  print("訓練集形狀：",x_train.shape,y_train.shape)
  print("驗證集形狀：",x_val.shape,y_val.shape)
  print("測試集形狀：",x_test.shape,y_test.shape)

  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
def LSTM_model(x_train, y_train, x_val, y_val):
  model = Sequential()
  model.add(LSTM(200, return_sequences=False, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
  early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
  history = model.fit(x_train, y_train, batch_size=200, epochs=100, validation_data=(x_val, y_val),callbacks=early_stop)

  return model

In [ ]:
def CNN_LSTM_model(x_train, y_train, x_val, y_val):
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=3, strides=1,padding='same', activation="relu"))
  model.add(Conv1D(filters=64, kernel_size=3, strides=1,padding='same' ,activation="relu"))
  model.add(MaxPooling1D(pool_size=2, strides=1))
  model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation="relu"))
  model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation="relu"))
  model.add(MaxPooling1D(pool_size=3, strides=1))
  model.add(LSTM(256, return_sequences=True, activation='relu'))
  model.add(LSTM(128, return_sequences=False, activation='relu'))
  model.add(Dense(1))

  model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
  early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)
  history = model.fit(x_train, y_train, batch_size=200, epochs=100, validation_data=(x_val, y_val),callbacks=early_stop)

  return model

In [ ]:
def score(model,test_X,test_Y):
  # predict probabilities for test set
  predicted = model.predict(test_X)

  #模型評價
  print("Mean Absolute Error:", mean_absolute_error(test_Y, predicted))
  print('Mean Squared Error:', mean_squared_error(test_Y, predicted))
  print('Root Mean Squared Error:', sqrt(mean_squared_error(test_Y, predicted)))
  print("Coefficient of Determination:", r2_score(test_Y, predicted))

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(24,7)

x_model = CNN_LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18914, 24, 7) (18914,)
驗證集形狀： (4711, 24, 7) (4711,)
測試集形狀： (2607, 24, 7) (2607,)
Epoch 1/100
95/95 [==============================] - 49s 452ms/step - loss: 0.0167 - mse: 0.0167 - mae: 0.0928 - val_loss: 0.0047 - val_mse: 0.0047 - val_mae: 0.0502
Epoch 2/100
95/95 [==============================] - 42s 444ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0308 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0315
Epoch 3/100
95/95 [==============================] - 43s 455ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0241 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0281
Epoch 4/100
95/95 [==============================] - 43s 457ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0234 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0282
Epoch 5/100
95/95 [==============================] - 41s 430ms/step - loss: 9.6226e-04 - mse: 9.6226e-04 - mae: 0.0225 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0275
Epoch 6/100
95/95 [==============================] - 42s 444ms/step - los

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(36,7)

x_model = CNN_LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18902, 36, 7) (18902,)
驗證集形狀： (4699, 36, 7) (4699,)
測試集形狀： (2595, 36, 7) (2595,)
Epoch 1/100
95/95 [==============================] - 79s 783ms/step - loss: 0.0194 - mse: 0.0194 - mae: 0.1031 - val_loss: 0.0077 - val_mse: 0.0077 - val_mae: 0.0641
Epoch 2/100
95/95 [==============================] - 67s 712ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0416 - val_loss: 0.0026 - val_mse: 0.0026 - val_mae: 0.0382
Epoch 3/100
95/95 [==============================] - 73s 767ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0278 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0355
Epoch 4/100
95/95 [==============================] - 68s 720ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0238 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0301
Epoch 5/100
95/95 [==============================] - 69s 725ms/step - loss: 9.3710e-04 - mse: 9.3710e-04 - mae: 0.0223 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0276
Epoch 6/100
95/95 [==============================] - 67s 709ms/step - los

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(48,7)

x_model = CNN_LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18890, 48, 7) (18890,)
驗證集形狀： (4687, 48, 7) (4687,)
測試集形狀： (2583, 48, 7) (2583,)
Epoch 1/200
189/189 [==============================] - 45s 229ms/step - loss: 0.0104 - mse: 0.0104 - mae: 0.0718 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0528
Epoch 2/200
189/189 [==============================] - 44s 233ms/step - loss: 0.0027 - mse: 0.0027 - mae: 0.0394 - val_loss: 0.0031 - val_mse: 0.0031 - val_mae: 0.0411
Epoch 3/200
189/189 [==============================] - 44s 235ms/step - loss: 0.0019 - mse: 0.0019 - mae: 0.0325 - val_loss: 0.0027 - val_mse: 0.0027 - val_mae: 0.0393
Epoch 4/200
189/189 [==============================] - 45s 237ms/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0291 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0353
Epoch 5/200
189/189 [==============================] - 43s 226ms/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0268 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0346
Epoch 6/200
189/189 [==============================] - 45s 237ms/step -

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(72,7)

x_model = CNN_LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18866, 72, 7) (18866,)
驗證集形狀： (4663, 72, 7) (4663,)
測試集形狀： (2559, 72, 7) (2559,)
Epoch 1/200
189/189 [==============================] - 28s 130ms/step - loss: 0.0122 - mse: 0.0122 - mae: 0.0797 - val_loss: 0.0057 - val_mse: 0.0057 - val_mae: 0.0572
Epoch 2/200
189/189 [==============================] - 24s 128ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0424 - val_loss: 0.0036 - val_mse: 0.0036 - val_mae: 0.0454
Epoch 3/200
189/189 [==============================] - 24s 129ms/step - loss: 0.0023 - mse: 0.0023 - mae: 0.0357 - val_loss: 0.0040 - val_mse: 0.0040 - val_mae: 0.0473
Epoch 4/200
189/189 [==============================] - 24s 129ms/step - loss: 0.0019 - mse: 0.0019 - mae: 0.0323 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.0368
Epoch 5/200
189/189 [==============================] - 24s 127ms/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0292 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0362
Epoch 6/200
189/189 [==============================] - 24s 128ms/step -

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(96,7)

x_model = CNN_LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18842, 96, 7) (18842,)
驗證集形狀： (4639, 96, 7) (4639,)
測試集形狀： (2535, 96, 7) (2535,)
Epoch 1/200
189/189 [==============================] - 37s 183ms/step - loss: 0.0137 - mse: 0.0137 - mae: 0.0827 - val_loss: 0.0102 - val_mse: 0.0102 - val_mae: 0.0763
Epoch 2/200
189/189 [==============================] - 34s 177ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0428 - val_loss: 0.0046 - val_mse: 0.0046 - val_mae: 0.0515
Epoch 3/200
189/189 [==============================] - 34s 178ms/step - loss: 0.0021 - mse: 0.0021 - mae: 0.0348 - val_loss: 0.0034 - val_mse: 0.0034 - val_mae: 0.0436
Epoch 4/200
189/189 [==============================] - 32s 172ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0313 - val_loss: 0.0025 - val_mse: 0.0025 - val_mae: 0.0377
Epoch 5/200
189/189 [==============================] - 33s 176ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0296 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0349
Epoch 6/200
189/189 [==============================] - 33s 177ms/step -

# LSTM

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(24,7)

x_model = LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18914, 24, 7) (18914,)
驗證集形狀： (4711, 24, 7) (4711,)
測試集形狀： (2607, 24, 7) (2607,)
Epoch 1/100
95/95 [==============================] - 18s 170ms/step - loss: 0.0135 - mse: 0.0135 - mae: 0.0855 - val_loss: 0.0073 - val_mse: 0.0073 - val_mae: 0.0630
Epoch 2/100
95/95 [==============================] - 17s 175ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0417 - val_loss: 0.0027 - val_mse: 0.0027 - val_mae: 0.0386
Epoch 3/100
95/95 [==============================] - 18s 184ms/step - loss: 0.0018 - mse: 0.0018 - mae: 0.0320 - val_loss: 0.0020 - val_mse: 0.0020 - val_mae: 0.0328
Epoch 4/100
95/95 [==============================] - 16s 168ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0272 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.0327
Epoch 5/100
95/95 [==============================] - 16s 167ms/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0262 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0301
Epoch 6/100
95/95 [==============================] - 18s 190ms/step - loss: 0.001

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(36,7)

x_model = LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18902, 36, 7) (18902,)
驗證集形狀： (4699, 36, 7) (4699,)
測試集形狀： (2595, 36, 7) (2595,)
Epoch 1/100
95/95 [==============================] - 9s 81ms/step - loss: 0.0152 - mse: 0.0152 - mae: 0.0905 - val_loss: 0.0088 - val_mse: 0.0088 - val_mae: 0.0694
Epoch 2/100
95/95 [==============================] - 6s 59ms/step - loss: 0.0036 - mse: 0.0036 - mae: 0.0455 - val_loss: 0.0028 - val_mse: 0.0028 - val_mae: 0.0399
Epoch 3/100
95/95 [==============================] - 7s 72ms/step - loss: 0.0020 - mse: 0.0020 - mae: 0.0340 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.0362
Epoch 4/100
95/95 [==============================] - 6s 58ms/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0287 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0294
Epoch 5/100
95/95 [==============================] - 7s 71ms/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0267 - val_loss: 0.0022 - val_mse: 0.0022 - val_mae: 0.0354
Epoch 6/100
95/95 [==============================] - 6s 59ms/step - loss: 0.0011 - mse: 0.0

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(48,7)

x_model = LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18890, 48, 7) (18890,)
驗證集形狀： (4687, 48, 7) (4687,)
測試集形狀： (2583, 48, 7) (2583,)
Epoch 1/100
95/95 [==============================] - 11s 91ms/step - loss: 0.0130 - mse: 0.0130 - mae: 0.0814 - val_loss: 0.0050 - val_mse: 0.0050 - val_mae: 0.0523
Epoch 2/100
95/95 [==============================] - 7s 73ms/step - loss: 0.0027 - mse: 0.0027 - mae: 0.0393 - val_loss: 0.0034 - val_mse: 0.0034 - val_mae: 0.0428
Epoch 3/100
95/95 [==============================] - 8s 85ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0301 - val_loss: 0.0025 - val_mse: 0.0025 - val_mae: 0.0371
Epoch 4/100
95/95 [==============================] - 7s 73ms/step - loss: 0.0016 - mse: 0.0016 - mae: 0.0295 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0308
Epoch 5/100
95/95 [==============================] - 8s 87ms/step - loss: 0.0012 - mse: 0.0012 - mae: 0.0256 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0294
Epoch 6/100
95/95 [==============================] - 8s 87ms/step - loss: 0.0011 - mse: 0.

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(72,7)

x_model = LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18866, 72, 7) (18866,)
驗證集形狀： (4663, 72, 7) (4663,)
測試集形狀： (2559, 72, 7) (2559,)
Epoch 1/100
95/95 [==============================] - 14s 120ms/step - loss: 0.0126 - mse: 0.0126 - mae: 0.0808 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0687
Epoch 2/100
95/95 [==============================] - 11s 119ms/step - loss: 0.0027 - mse: 0.0027 - mae: 0.0389 - val_loss: 0.0036 - val_mse: 0.0036 - val_mae: 0.0450
Epoch 3/100
95/95 [==============================] - 12s 124ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0308 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0357
Epoch 4/100
95/95 [==============================] - 12s 123ms/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0271 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0296
Epoch 5/100
95/95 [==============================] - 11s 121ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0242 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.0379
Epoch 6/100
95/95 [==============================] - 12s 123ms/step - loss: 0.001

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = dataset(96,7)

x_model = LSTM_model(x_train, y_train, x_val, y_val)

score(x_model,x_test, y_test)

訓練集形狀： (18842, 96, 7) (18842,)
驗證集形狀： (4639, 96, 7) (4639,)
測試集形狀： (2535, 96, 7) (2535,)
Epoch 1/100
95/95 [==============================] - 18s 164ms/step - loss: 0.0133 - mse: 0.0133 - mae: 0.0826 - val_loss: 0.0059 - val_mse: 0.0059 - val_mae: 0.0575
Epoch 2/100
95/95 [==============================] - 15s 160ms/step - loss: 0.0028 - mse: 0.0028 - mae: 0.0403 - val_loss: 0.0028 - val_mse: 0.0028 - val_mae: 0.0390
Epoch 3/100
95/95 [==============================] - 15s 156ms/step - loss: 0.0018 - mse: 0.0018 - mae: 0.0314 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0337
Epoch 4/100
95/95 [==============================] - 15s 157ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0275 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0358
Epoch 5/100
95/95 [==============================] - 14s 153ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0249 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0279
Epoch 6/100
95/95 [==============================] - 16s 170ms/step - loss: 0.001